In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from decpomdp import DecPOMDP
import Classes
from constant import Constants
import time
import os


In [ ]:
problems = {'dectiger','longFireFight','relay4',
            '2generals','oneDoor','prisoners',
            'boxPushing','GridSmall',
            'recycling','wirelessDelay','broadcastChannel'
            }

In [9]:
file_name = 'wirelessDelay'
planning_horizon = 3
num_iterations = 1


In [10]:
#import problem
problem = DecPOMDP("wirelessDelay", 1,horizon=planning_horizon)
# constant = Constants(problem)
# utilities = Utilities(constant)
# Classes.set_problem(problem)

# # print(f"game of initiated with SOTA set to = {sota_} with horizon {planning_horizon}")
print("GAME DESCRIPTION :")
print(f"game size :\n\t|S| = {len(problem.states)}")
print(f"\t|Z| = {problem.num_joint_observations}\n\t|U| = {problem.num_joint_actions} with |U_i| = {problem.num_actions[0]}")
print(f"intiial_belief : {problem.b0}")


print(f"{problem.num_actions[0]},{problem.num_actions[0]}")

GAME DESCRIPTION :
game size :
	|S| = 64
	|Z| = 36
	|U| = 4 with |U_i| = 2
intiial_belief : [0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.  ]
2,2


In [10]:
# solve
def SOLVE(game,iterations):
    start_time = time.time()
    policy,leader_values,follower_values = game.solve(iterations,0.9)
    end_time = time.time()
    solve_time = end_time - start_time
    return policy,leader_values,follower_values,solve_time,


In [11]:
def initialize_database():
    database = {"gametype":[],
                "SOTA" : [],
                "horizon": [],
                    "num_iterations" : [],
                    "average_time" : [],
                    "number_of_beliefs" : [],
                    "ave_leader_value_b0":[],
                    "ave_follower_value_b0":[]
                    # "density" = []
                    # "gap":[]
                   
                    }
    return database

def add_to_database(database,horizon,game_type,num_iterations,average_time,num_beliefs,V0_B0,V1_B0,SOTA):
    sota = {True:"State of the Art" , False:"Stackelberg"}
    database["gametype"].append(game_type)
    database["horizon"].append(horizon)
    database["SOTA"].append(sota[SOTA])
    database["num_iterations"].append(num_iterations)
    database["average_time"].append(average_time)
    database["number_of_beliefs"].append(num_beliefs)
    database["ave_leader_value_b0"].append(V0_B0)
    database["ave_follower_value_b0"].append(V1_B0)
    # database["gap"].append(abs(V0_B0-V1_B0))
    # database["density"].append(density)
    return


In [12]:
database = initialize_database()
problem = DecPOMDP(file_name,1,horizon=planning_horizon)
for gametype in ["cooperative","zerosum","stackelberg"]:
    for horizon in range(1,planning_horizon+1):
        for sota_ in [False,True]:
            print(f"\n============= {gametype} GAME WITH HORIZON {horizon} , SOTA {sota_} ===========")
            game = Classes.PBVI(problem=problem.set_horizon(horizon),horizon=horizon,density=0.1,gametype=gametype,sota=sota_)
            policy,leader_values,follower_values, time_  = SOLVE(game,num_iterations)
            num_beliefs = game.belief_space.belief_size()
            for iters in range(num_iterations):
                add_to_database(database,horizon,gametype,iters+1,time_,num_beliefs,leader_values[iters],follower_values[iters],sota_)
print("Calculations done... exporting to csv....")
#POLICY PRINTING: 
# print("\nLEADER POLICY\n")
# policy[0].print_trees()
# print("\nFOLLOWER POLICY\n")
# policy[1].print_trees()        



database = pd.DataFrame(database)
path = "Results/"
file_name = f"{file_name}_{horizon}_{num_iterations}"
database.to_csv(file_name, index=False)




============= cooperative GAME WITH HORIZON 1 , SOTA False ===========
PROBLEM SET T0 ==> relay4
	belief expansion done
iteration : 1


	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done

============= cooperative GAME WITH HORIZON 1 , SOTA True ===========
PROBLEM SET T0 ==> relay4
	belief expansion done
iteration : 1
	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done

============= cooperative GAME WITH HORIZON 2 , SOTA False ===========
PROBLEM SET T0 ==> relay4
	belief expansion done
iteration : 1
	backup at timestep 2 done
	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done

============= cooperative GAME WITH HORIZON 2 , SOTA True ===========
PROBLEM SET T0 ==> relay4
	belief expansion done
iteration : 1
	backup at timestep 2 done
	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done

============= cooperative GAME WITH HORIZON 3 , SOTA False ===========
PROBLEM SET T0 ==> relay4
	belief expansion done
iteration : 1
	backup at timestep 3 done
	backup at timestep 2 done
	backup at timestep 1 done
	backup at 

In [13]:
database

,gametype,SOTA,horizon,num_iterations,average_time,number_of_beliefs,ave_leader_value_b0,ave_follower_value_b0
0,cooperative,Stackelberg,1,1,0.403922,6,-51.8,-51.8
1,cooperative,State of the Art,1,1,0.070940,6,-51.8,-51.8
2,cooperative,Stackelberg,2,1,1.611761,25,-53.6,-53.6
3,cooperative,State of the Art,2,1,0.224219,25,-53.6,-53.6
4,cooperative,Stackelberg,3,1,2.396509,46,-55.4,-55.4
5,cooperative,State of the Art,3,1,0.456918,46,-55.4,-55.4
6,zerosum,Stackelberg,1,1,0.279062,6,-51.8,51.8
7,zerosum,State of the Art,1,1,0.239747,6,-50.2,-50.2
8,zerosum,Stackelberg,2,1,1.397104,25,-53.6,53.6
9,zerosum,State of the Art,2,1,0.986555,25,-50.4,-50.4


game.value_function

In [14]:
for i in game.value_function.vector_sets[0]:
    print(i.get_value(game.belief_space.get_inital_belief()))

(-50.599999999999994, -47.0)


In [15]:
print(f"RESULTS FOR {file_name}:\n")

RESULTS FOR relay4_3_1:

